<a href="https://colab.research.google.com/github/bahador1/Monte-Carlo-Simulation-for-Shap/blob/main/monteCarloSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# What is Monte Carlo simulation?

- Monte Carlo simulation is, at its core, a simple technique that uses evolving random sampling to simulate probablty deterministic problems.

- Imagine your friend Bob offers you to participate in a game where he throws ten dice simultaneously. If the sum of the dice is between forty and fifty, he gives you ten dollars; else, you have to pay him two dollars.

- Should you play this game?

- To decide whether to play, you should first figure out the probabilities of winning some money. Yes, you can compute the likelihood of winning the game by hand; however, why struggle with some pen and paper when you can use the computer to solve the problem?



In [2]:
def throw_and_sum_n_dice(n):
    '''
    This function throws n dice and sums the results
    '''
    total = 0
    for i in range(0,n):
        throw = randint(1,6)
        total += throw
    return total